In [1]:
import torch

from transformers import AutoTokenizer, AutoModelWithLMHead

In [2]:
tokenizer = AutoTokenizer.from_pretrained('model-output/model-4-21-23/')
model = AutoModelWithLMHead.from_pretrained('model-output/model-4-21-23/', torch_dtype=torch.float32)

c:\Users\milan\miniconda3\envs\tfgpu\lib\site-packages\transformers\models\auto\modeling_auto.py:1338: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [3]:
# input_text = 'Magnetic declination is' # (good) top_k=1
# input_text = 'The IGRM is' # (hallucinations) top_k=1
# input_text = 'In a total solar eclipse' # (good) top_k=5 
# input_text = 'What is a spherical harmonic model?' # (ok) top_k=1, typical_p=0.8
# input_text = "A compass needle points towards" # (bad)
# input_text = "To correct my compass bearing, I need to know" # (good) top_k=50, typical_p=0.8
# input_text = "Geomagnetic calculator computes" (good but not great) top_k=50, typical_p=0.8
input_text = "The ionosphere is"

capacity = 15
generation_length = len(input_text.split()) * capacity * 2
generation_text = tokenizer.encode(input_text, return_tensors='pt')

In [4]:
print(model.num_parameters())

354823168


In [4]:
response = model.generate(
    input_ids=generation_text, 
    max_length=generation_length, 
    do_sample=True, 
    early_stopping=True,
    num_beams=10,
    no_repeat_ngram_size=4,
    top_k=50, 
    typical_p=0.9,
    temperature=0.9)

response = tokenizer.decode(response[0], skip_special_tokens=True)
response = response.replace("\n", "</EOL> ")

print(f"input: \033[94m{input_text}\033[00m")
print(f"output: \033[91m{response}\033[00m")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input: The ionosphere is
output: The ionosphere is a highly dynamic region with a large number of currents and interactions with the Earth’s magnetic field, known as space weather.,</EOL>  There are two main regions of the ionosphere. The outermost layer, the plasmasphere, is the source of the strongest currents in
